<a href="https://colab.research.google.com/github/Meow-nah-ngo/LPR/blob/main/LPR_Kwan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  **YoloV8**

In [ ]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 901.3/901.3 kB 20.2 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO
# Load a model
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 118MB/s]


#Easyocr

In [ ]:
pip install easyocr


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.2/912.2 kB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 26.4 MB/s eta 0:00:00


#Other pip

In [ ]:
pip install opencv-python

In [ ]:
pip install fuzzywuzzy


In [15]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="wGem50Rjvyol0TlOkRk7")
project = rf.workspace("p-opz7u").project("l-abonw")
version = project.version(2)
dataset = version.download("yolov8")
#Dataset from roboflow

loading Roboflow workspace...
loading Roboflow project...


In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="wGem50Rjvyol0TlOkRk7")
project = rf.workspace("ai-888").project("license-plate-y3o6h")
version = project.version(1)
dataset = version.download("yolov8")
#Dataset from roboflow

loading Roboflow workspace...
loading Roboflow project...


#Model

In [22]:
import easyocr
import os
import cv2
import re
import csv
from collections import defaultdict

# EasyOCR Reader (ใช้ภาษาไทย)
reader = easyocr.Reader(['th'], gpu=True)

# โฟลเดอร์รูปภาพ
folder_paths = ['l-2/train/images', 'license-plate-1/train/images']
results = defaultdict(list)

# ลิสต์จังหวัด (ไม่มีซ้ำ)
provinces_list = list(set([
    "กระบี่", "กรุงเทพมหานคร", "กาญจนบุรี", "กาฬสินธุ์", "กำแพงเพชร", "ขอนแก่น", "จันทบุรี",
    "ฉะเชิงเทรา", "ชลบุรี", "ชัยนาท", "ชัยภูมิ", "ชุมพร", "เชียงราย", "เชียงใหม่",
    "ตรัง", "ตราด", "ตาก", "นครนายก", "นครปฐม", "นครพนม", "นครราชสีมา", "นครศรีธรรมราช",
    "นครสวรรค์", "นนทบุรี", "นราธิวาส", "น่าน", "บึงกาฬ", "บุรีรัมย์", "ปทุมธานี",
    "ประจวบคีรีขันธ์", "ปราจีนบุรี", "ปัตตานี", "พระนครศรีอยุธยา", "พะเยา", "พังงา", "พัทลุง",
    "พิจิตร", "พิษณุโลก", "เพชรบุรี", "เพชรบูรณ์", "แพร่", "ภูเก็ต", "มหาสารคาม", "มุกดาหาร",
    "แม่ฮ่องสอน", "ยโสธร", "ยะลา", "ร้อยเอ็ด", "ระนอง", "ระยอง", "ราชบุรี", "ลพบุรี",
    "ลำปาง", "ลำพูน", "ศรีสะเกษ", "สกลนคร", "สงขลา", "สตูล", "สมุทรปราการ",
    "สมุทรสงคราม", "สมุทรสาคร", "สระแก้ว", "สระบุรี", "สิงห์บุรี", "สุโขทัย",
    "สุพรรณบุรี", "สุราษฎร์ธานี", "สุรินทร์", "หนองคาย", "หนองบัวลำภู", "อำนาจเจริญ",
    "อุดรธานี", "อุตรดิตถ์", "อุทัยธานี", "อุบลราชธานี", "อ่างทอง"
]))

# ฟังก์ชันกรองข้อความไม่จำเป็น
def clean_text(text):
    text = text.strip()
    return text if len(text) > 1 and text not in ['.', '!', '#', 'ๆ', '…'] else None

# ฟังก์ชันตรวจป้ายทะเบียนไทย
def is_license_plate(text):
    return bool(re.match(r'^[0-9]{1,2}\s?[ก-ฮA-Za-z]{1,2}\s?[0-9]{1,4}$', text))

# ฟังก์ชันตรวจจังหวัดไทย
def is_province(text):
    return any(province in text for province in provinces_list)

# เริ่มวนลูปอ่านภาพ
for folder_path in folder_paths:
    for filename in os.listdir(folder_path):
        if filename.endswith(('.jpg', '.png')):
            image_path = os.path.join(folder_path, filename)

            # โหลดและปรับภาพ
            image = cv2.imread(image_path)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            adjusted = cv2.convertScaleAbs(gray, alpha=1.5, beta=0)

            # อ่านข้อความจากภาพ
            result = reader.readtext(adjusted, contrast_ths=0.05, text_threshold=0.4)

            # แสดงผลก่อนกรอง
            print(f"\n--- Results for {filename} ---")
            for box, text, conf in result:
                print(f"Detected: {text}")

            # กรองข้อความ
            cleaned = [clean_text(text) for _, text, _ in result if clean_text(text)]
            results[filename] = cleaned

# จัดระเบียบข้อมูล
organized_data = []

for filename, texts in results.items():
    plates = list(filter(is_license_plate, texts))
    provinces = list(filter(is_province, texts))

    for plate in plates:
        province_match = provinces[0] if provinces else ''
        organized_data.append({
            'filename': filename,
            'license_plate': plate,
            'province': province_match
        })

# แสดงผล
print("\n--- Organized Data ---")
if organized_data:
    for data in organized_data:
        print(f"File: {data['filename']}")
        print(f"  License Plate: {data['license_plate']}")
        print(f"  Province: {data['province']}")
else:
    print("No valid license plates and provinces detected.")

# บันทึกเป็น CSV
with open('organized_license_data.csv', mode='w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['filename', 'license_plate', 'province'])
    writer.writeheader()
    writer.writerows(organized_data)

เอาต์พุตของการสตรีมมีการตัดเหลือเพียง 5000 บรรทัดสุดท้าย
--- Results for 0-43568000_1771409379652673_5214585653102116864_n_jpg.rf.4e52eb7801a8b93d87416f3fa163d5bc.jpg ---
Detected: 1ผบ2992
Detected: กรแททมหานคร

--- Results for 0-69905637_2267958876664385_6997511867132280832_n_jpg.rf.a3e38f6b283d4c06da83ffe8dc5d80c1.jpg ---
Detected: งย 3778
Detected: ชลบรี

--- Results for 0-245804961_4249451675181752_7652365658550728902_n_jpg.rf.cfe46a4c9a71c0eb3667e970681a3cca.jpg ---
Detected: 1ขฐ6158
Detected: กรุงเทพมหานคร

--- Results for 0-243709024_4205815956211991_3246026150760081623_n_jpg.rf.ce1b32aa0eeb069b009d6797aea1cb37.jpg ---
Detected: 1ขพ7917
Detected: กรุงเทพมหานร

--- Results for 0-83827521_2572170012909935_1392750489845628928_n_jpg.rf.45068fec0d37c096d5f884bdac93dd7b.jpg ---
Detected: 6กฆ5795
Detected: กรงเทพมหานคร

--- Results for 0-71270564_2291233887670217_7889521117893754880_n_jpg.rf.a86ddc8d4b91301d5bb226d04b48c0fb.jpg ---
Detected: 1กล 1974
Detected: กรงเทพมหานคร

--- Results

In [24]:
import os

for path in folder_paths:
    print(path, "=>", os.path.exists(path))


l-2/train/images => True


In [23]:
!pip install gspread oauth2client
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [ ]:
from flask import Flask, request, jsonify
import cv2
import easyocr
import re
import numpy as np
import requests # เพิ่มไลบรารี requests เพื่อส่งข้อมูลไป Arduino
app = Flask(__name__)
# EasyOCR setup
reader = easyocr.Reader(['th'], gpu=True)
# ฟังก์ชันกรองข้อมูลที่ไม่จำเป็น
def clean_text(text):
    return text.strip() if len(text.strip()) > 1 and text.strip() not in ['.', '!', '#', 'ๆ', '…'] else None
# ฟังก์ชันตรวจจับเลขทะเบียนรถ
def is_license_plate(text):
    return bool(re.match(r'^[0-9]{1,2}\s?[A-Za-zก-ฮ]{1,2}\s?[0-9]{1,4}$', text))
# ฟังก์ชันตรวจจับจังหวัด
def is_province(text):
    provinces = [
        "ขอนแก่น", "จันทบุรี", "ฉะเชิงเทรา", "ชลบุรี", "ชัยนาท", "ชัยภูมิ", "ชุมพร", "กรุงเทพมหานคร",
        "เชียงใหม่", "เชียงราย", "ตรัง", "ตราด", "ตาก", "นครนายก", "นครปฐม", "นครพนม", "นครราชสีมา", "นครศรีธรรมราช",
        "นครสวรรค์", "นราธิวาส", "น่าน", "บึงกาฬ", "บุรีรัมย์", "ปทุมธานี", "ประจวบคีรีขันธ์", "ปราจีนบุรี", "ปัตตานี",
        "พะเยา", "พังงา", "พัทลุง", "พิจิตร", "พิษณุโลก", "เพชรบูรณ์", "เพชรบุรี", "ภูเก็ต", "มุกดาหาร",
        "ยโสธร", "ระยอง", "ราชบุรี", "ลำปาง", "ลำพูน", "เลย", "ศรีสะเกษ", "สกลนคร", "สมุทรปราการ", "สมุทรสงคราม",
        "สมุทรสาคร", "สระแก้ว", "สระบุรี", "สิงห์บุรี", "สุพรรณบุรี", "สุราษฎร์ธานี", "สุรินทร์", "สงขลา", "สุโขทัย",
        "สตูล", "อำนาจเจริญ", "อุดรธานี", "อุตรดิตถ์", "อุทัยธานี", "อ่างทอง", "อุบลราชธานี", "สมุทรปราการ",
        "ปทุมธานี", "นราธิวาส", "ประจวบคีรีขันธ์", "ราชบุรี", "กระบี่", "มหาสารคาม", "หนองคาย", "สระแก้ว", "ชัยภูมิ"
    ]
    return any(province.lower() in text.lower() for province in provinces)
# ฟังก์ชันส่งข้อมูลไปยัง Arduino API
def send_to_arduino(license_plate, province, image_link):
    arduino_url = "http://arduino.local/api" # URL ของ Arduino ที่รับข้อมูล
    data = {
        'license_plate': license_plate,
        'province': province,
        'image_link': image_link
    }
    try:
        response = requests.post(arduino_url, json=data)
        if response.status_code == 200:
            return "Data sent to Arduino successfully!"
        else:
            return f"Failed to send data to Arduino, status code: {response.status_code}"
    except Exception as e:
        return f"Error sending data to Arduino: {e}"
@app.route('/process_image', methods=['POST'])
def process_image():
    # รับข้อมูลภาพจาก request
    if 'image' not in request.files:
        return jsonify({"error": "No image provided"}), 400
    file = request.files['image']
    image = np.frombuffer(file.read(), np.uint8)
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    # แปลงภาพเป็นขาวดำ
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # ปรับคอนทราสต์
    adjusted_image = cv2.convertScaleAbs(gray_image, alpha=1.5, beta=0)
    # ใช้ EasyOCR กับภาพที่ปรับแต่งแล้ว
    result = reader.readtext(adjusted_image, contrast_ths=0.05, text_threshold=0.4)
    # กรองข้อความที่ไม่ต้องการ
    cleaned_result = [x[1].strip() for x in result if clean_text(x[1])]
    # ตรวจจับทะเบียนรถและจังหวัด
    license_plate = None
    province = None
    image_link = "http://example.com/image.jpg" # ตัวอย่างลิงก์ภาพ
    for text in cleaned_result:
        if is_license_plate(text):
            license_plate = text
        elif is_province(text):
            province = text
    # สร้างผลลัพธ์ที่ต้องการ
    response = {"license_plate": license_plate, "province": province}
    # ส่งข้อมูลไปยัง Arduino API
    if license_plate and province:
        arduino_response = send_to_arduino(license_plate, province, image_link)
        print(arduino_response)
    return jsonify(response)
if __name__ == '__main__':
    app.run(host="0.0.0.0", port=8080)

ModuleNotFoundError: No module named 'easyocr'